In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

def compute_ret(df):
    log_ret = df['Adj Close'].apply(np.log).diff().fillna(0).to_frame()
    sim_ret = df['Adj Close'].pct_change().fillna(0).to_frame()
    log_ret.columns = ['Log Ret']
    sim_ret.columns = ['Simple Ret']
    return pd.concat([df, log_ret, sim_ret], axis=1)

df_aapl = pd.read_csv(r'AAPL.csv', index_col='Date').iloc[0:-4]
df_aapl.index = pd.to_datetime(df_aapl.index, format='%Y-%m-%d')

df_ibm = pd.read_csv(r'IBM.csv', index_col='Date').iloc[0:-4]
df_ibm.index = pd.to_datetime(df_ibm.index, format='%Y-%m-%d')

df_jpm = pd.read_csv(r'JPM.csv', index_col='Date').iloc[0:-4]
df_jpm.index = pd.to_datetime(df_jpm.index, format='%Y-%m-%d')

df_dji = pd.read_csv(r'DJI.csv', index_col='Date').iloc[0:-4]
df_dji.index = pd.to_datetime(df_dji.index, format='%Y-%m-%d')

df_aapl = compute_ret(df_aapl)
df_ibm = compute_ret(df_ibm)
df_jpm = compute_ret(df_jpm)
df_dji = compute_ret(df_dji)

In [11]:
#question1

from sklearn.linear_model import LinearRegression

def ols_linreg(X, y, y_norm):
    X = sm.add_constant(X)
    model = LinearRegression().fit(X, y)
    score = model.score(X, y_norm)
    return model, score

def tabulate_res(aapl_reg, aapl_reg_score, ibm_reg, ibm_reg_score, jpm_reg, jpm_reg_score):
    return pd.DataFrame({
        'coef': [aapl_reg.coef_[1], ibm_reg.coef_[1], jpm_reg.coef_[1]], 
        'intercept': [aapl_reg.intercept_, ibm_reg.intercept_, jpm_reg.intercept_],
        'score': [aapl_reg_score, ibm_reg_score, jpm_reg_score]
    }, index=['AAPL', 'IBM', 'JPM'])

In [12]:
aapl_linreg, aapl_linreg_score = ols_linreg(df_dji.iloc[1:]['Simple Ret'], df_aapl.iloc[1:]['Simple Ret'], df_aapl.iloc[1:]['Simple Ret'])
ibm_linreg, ibm_linreg_score = ols_linreg(df_dji.iloc[1:]['Simple Ret'], df_ibm.iloc[1:]['Simple Ret'], df_ibm.iloc[1:]['Simple Ret'])
jpm_linreg, jpm_linreg_score = ols_linreg(df_dji.iloc[1:]['Simple Ret'], df_jpm.iloc[1:]['Simple Ret'], df_jpm.iloc[1:]['Simple Ret'])

lin_reg_res = tabulate_res(aapl_linreg, aapl_linreg_score, ibm_linreg, ibm_linreg_score, jpm_linreg, jpm_linreg_score)
lin_reg_res

/Users/yeehonglow/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/yeehonglow/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/yeehonglow/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,coef,intercept,score
AAPL,1.327464,0.000047,0.518874
IBM,0.961929,-0.000468,0.418597
JPM,0.930549,-0.000377,0.556493


In [13]:
#question 2
from sklearn.linear_model import HuberRegressor

def huber_linreg(X, y, y_norm, epsilon=1.35):
    X = sm.add_constant(X)
    # epsilon, parameter that controls the number of samples that 
    # should be classified as outliers. The smaller the epsilon, the 
    # more robust it is to outliers
    model = HuberRegressor(epsilon=epsilon).fit(X, y)
    score = model.score(X, y_norm)
    return model, score

aapl_huber_reg, aapl_huber_score = huber_linreg(df_dji['Simple Ret'], df_aapl['Simple Ret'], df_aapl['Simple Ret'], epsilon=1.35)
ibm_huber_reg, ibm_huber_score = huber_linreg(df_dji['Simple Ret'], df_ibm['Simple Ret'], df_ibm['Simple Ret'], epsilon=1.35)
jpm_huber_reg, jpm_huber_score = huber_linreg(df_dji['Simple Ret'], df_jpm['Simple Ret'], df_jpm['Simple Ret'], epsilon=1.35)

huber_reg_res = tabulate_res(aapl_huber_reg, aapl_huber_score, ibm_huber_reg, ibm_huber_score, jpm_huber_reg, jpm_huber_score)
huber_reg_res

/Users/yeehonglow/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/yeehonglow/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/yeehonglow/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,coef,intercept,score
AAPL,1.272508,-0.000136,0.517697
IBM,0.976089,-0.000266,0.418488
JPM,0.918433,-0.000439,0.554913


In [15]:
def combine_abn_ret(df):
    df = df.copy()
    log_ret = df['Adj Close'].apply(np.log).diff().fillna(0).to_frame()
    sim_ret = df['Adj Close'].pct_change().fillna(0).to_frame()
    log_ret.columns = ['Abn Log Ret']
    sim_ret.columns = ['Abn Simple Ret']
    return pd.concat([df, log_ret, sim_ret], axis=1)

df_abn_aapl = combine_abn_ret(df_abn_aapl)
df_abn_ibm = combine_abn_ret(df_abn_ibm)
df_abn_jpm = combine_abn_ret(df_abn_jpm)

NameError: name 'df_abn_aapl' is not defined